In [1]:
from  PIL import Image
import PIL.Image
import redis
import PIL
import os
import cv2
# from module import k_mean_color_detection
import numpy as np
import random
from module import  process_raw_d_logs, process_raw_cc_logs, process_cc_logs

# Connect to the Redis server
r = redis.Redis(host='localhost', port=6379, db=0)

def clear_redis_database():
    r.flushdb()
    print("Redis database cleared.")

# Function to store string data with key namespacing
def store_string_with_namespace(serial_number, constant_string, value):
    full_key = f"{serial_number}:{constant_string}"
    r.set(full_key, value)
    print(f"Stored {full_key} -> {value}")

# Function to retrieve all data matching the constant part of the key
def retrieve_keys(constant_string):
    matching_keys = r.keys(f"*:{constant_string}")
    results = {}
    for key in matching_keys:
        value = r.get(key)
        if value:
            decoded_key = key.decode()
            decoded_value = value.decode()
            # np_arr = np.frombuffer(value, np.uint8)
            # Decode the numpy array back to an image
            # retrieved_image = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)
            # results[decoded_key] = retrieved_image
            results[decoded_key] = decoded_value
            # r.delete(decoded_key)
            # print(results)
            # r.delete(key)  # Delete the key after retrieving the value
            # print(f"Retrieved and deleted {decoded_key} -> {decoded_value}")
    return results

def retrieve_image(constant_string):
    matching_keys = r.keys(f"*:{constant_string}")
    results = {}
    for key in matching_keys:
        value = r.get(key)
        if value:
            decoded_key = key.decode()
            # decoded_value = value.decode()
            np_arr = np.frombuffer(value, np.uint8)
            # Decode the numpy array back to an image
            image = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)
            # image = image.reshape((image.shape[0] * image.shape[1], 3))
            # image = image.reshape((-1, image.shape[2]))
            # cv2.imwrite(f"/home/annone/ai/backend/stream/d_temp/{decoded_key}.png", image)
            results[decoded_key] = image
            # results[decoded_key] = decoded_value
            # r.delete(decoded_key)
            # print(results)
            # r.delete(key)  # Delete the key after retrieving the value
            # print(f"Retrieved and deleted {decoded_key} -> {decoded_value}")
    return results

# def process_raw_d_logs(constant_string):
#     while True:
#         matching_keys = r.keys(f"*:{constant_string}")
#         # print(matching_keys)
#         for key in matching_keys:
#             value = r.get(key)
#             decode_key = key.decode()
#             decode_value = value.decode().split("|")
#             track_id = decode_value[5]
#             image_path = decode_value[7]
#             try:
#                 image = cv2.imread(f"/home/annone/ai-camera/backend/stream/temp/{image_path}")
#                 image = image.reshape((image.shape[0] * image.shape[1], 3))
#                 aa = k_mean_color_detection(image)
#                 decode_value[7]  = aa
#                 r.set(f"{track_id}_{random.randint(0,9999)}:d_log","|".join(decode_value))
#                 os.remove(f"/home/annone/ai-camera/backend/stream/temp/{image_path}")
#                 r.delete(decode_key)
#             except:
#                 # print("excepted")
#                 continue
            # show_rgb_colors(aa,image)
            # print(decode_value)

# def process_d_logs(constant_string):
#     conn = Database.get_connection()
#     cursor = conn.cursor()
#     batch = []
#     while True:
#         matching_keys = r.keys(f"*:{constant_string}")
#         for key in matching_keys:
#             value = r.get(key)
#             decode_key = key.decode()
#             decode_value = value.decode().split("|")
#             batch.append(tuple(decode_value))
#         if len(batch) > 0:
#             query = 'INSERT INTO "DetectionLog" ("cameraId", "camera_ip", "timestamp", "boxCoords", "detectionClass", "trackId", "classConfidence", "metadata") VALUES %s;'
#             execute_values(cursor, query, batch)
#             conn.commit()
#             cursor.close()
#             conn.close()
#             batch = []

2024-09-16 01:40:46.927631: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-16 01:40:46.940268: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-16 01:40:46.944035: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-16 01:40:46.953907: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-16 01:40:47.731128: W tensorflow/compiler/tf2

{'dbname': 'logs', 'user': 'root', 'password': 'team123', 'host': '34.47.148.81', 'port': '8080'}
start


/home/annone/ai/backend/stream/module.py:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  crowd_count_model.load_state_dict(torch.load(PATH_model, map_location=device))


In [ ]:
retrieve_image("image")["817_qcYXs5PM_39_1724913805_127_0_0_1_976:image"]

In [ ]:
retrieve_keys("raw_d_log")

In [2]:
clear_redis_database()

Redis database cleared.


In [ ]:
import torch
torch.cuda.empty_cache()

In [1]:
import redis

# Connect to Redis
r = redis.Redis(host='localhost', port=6379, db=0)

# Get all keys from Redis
keys = r.keys('*')

# Loop through the keys and get the data for each key
for i, key in enumerate(keys):
    if i ==10:
        break
    # Decode key to get human-readable format
    decoded_key = key.decode('utf-8')
    
    # Get the value associated with the key
    value = r.get(key)
    
    # Decode the value, if it's stored as a string
    decoded_value = value.decode('utf-8') if value else None
    
    # Print the key-value pair
    print(f"Key: {decoded_key}, Value: {decoded_value}")


Key: motorbike-rider_0.91_5b3e1275-f2da-483e-a51a-ba415c92d4cc, Value: {'custom_track_id': 'motorbike-rider_0.91_5b3e1275-f2da-483e-a51a-ba415c92d4cc', 'camera_id': 2, 'camera_ip': '198.78.45.89', 'first_appearance': 1726440941.3397026, 'last_appearance': 1726440941.3397026, 'dbbox': [[220, 34, 250, 78]], 'dlabel': ['motorbike-rider'], 'dconf': [0.9110313653945923]}
Key: pickup_0.89_c3a3316b-35cd-4032-a566-9f5252e499a5, Value: {'custom_track_id': 'pickup_0.89_c3a3316b-35cd-4032-a566-9f5252e499a5', 'camera_id': 2, 'camera_ip': '198.78.45.89', 'first_appearance': 1726440943.2046707, 'last_appearance': 1726440943.350726, 'dbbox': [[31, 129, 49, 174], [33, 128, 52, 174], [34, 128, 52, 174], [37, 125, 54, 169], [38, 124, 55, 168]], 'dlabel': ['pickup', 'car', 'auto', 'scooty-rider', 'car'], 'dconf': [0.8942528963088989, 0.5527427196502686, 0.551765501499176, 0.6766818761825562, 0.5470042824745178]}
Key: motorbike-rider_0.60_9fad54b6-0e1a-4931-a54f-c6e978bd997a, Value: {'custom_track_id': 'm

## Boundig Box

In [ ]:
import cv2
import numpy as np
import torch
from ultralytics import YOLO
from sort.sort import Sort  # Example of an external tracking library
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = YOLO("/home/annone/ai/models/objseg50e.pt")
model.to(device)
print(device)

tracker = Sort()

# Function to process frames in batches
def process_frame_batch(frames):
    resized_frames = [cv2.resize(frame, (640 // 32 * 32, 480 // 32 * 32)) for frame in frames]

    frames_tensor = torch.from_numpy(np.stack(resized_frames)).permute(0, 3, 1, 2).float().to(device) / 255.0

    with torch.no_grad():
        batch_results = model(frames_tensor, device=device)

    return batch_results, resized_frames

def track_objects(frames, batch_results):
    tracked_frames = []
    
    for frame, result in zip(frames, batch_results):
        detections = []
        
        if result.boxes:
            for box in result.boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                score = box.conf[0].item()
                label = model.names[int(box.cls[0])]
                
                detections.append([x1, y1, x2, y2, score])

        tracks = tracker.update(np.array(detections))

        for track in tracks:
            track_id = int(track[4])
            x1, y1, x2, y2 = map(int, track[:4])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"ID: {track_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        tracked_frames.append(frame)
    
    return tracked_frames

def stream_process(camera_id, camera_ip, video_path, batch_size=8):
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Error opening video file: {video_path}")
        return

    frames = []
    t1 = time.time()
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frames.append(frame)

        if len(frames) >= batch_size:
            batch_results, resized_frames = process_frame_batch(frames)

            tracked_frames = track_objects(resized_frames, batch_results)

            for tracked_frame in tracked_frames:
                cv2.imshow("Tracked Frame", tracked_frame)
            frames = []
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    t2 = time.time()
    print(t2-t1)
    cap.release()
    cv2.destroyAllWindows()

video_path = '/home/annone/ai/data/T-pole wrong way.mp4'
cam_ip = '127.0.0.1'
cam_id = "1"
stream_process(cam_id, cam_ip, video_path, batch_size=2)


## Segmention

In [1]:
import cv2
import numpy as np
import torch
from ultralytics import YOLO
from sort.sort import Sort
import time
# from module import generate_custom_string
from collections import defaultdict
import math
import redis

# Initialize device and model
PARENT_DIR = "/home/annone/ai"
r = redis.Redis(host='localhost', port=6379, db=0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = YOLO("/home/annone/ai/models/objseg50e.pt")
model.to(device)
print(f"{device} as Computation Device initiated")
tracker = Sort()

# CONSTANTS PER CAMERA
width, height = 640,480
camera_ip = "198.78.45.89"
camera_id = 2
fps = 30
class_list = ['auto', 'bike-rider', 'bolero', 'bus', 'car', 'hatchback', 'jcb', 'motorbike-rider', 'omni', 'pickup',
              'scooty-rider', 'scorpio', 'sedan', 'suv', 'swift', 'thar', 'tractor', 'truck', 'van']
previous_positions = defaultdict(lambda: {"x": 0, "y": 0, "time": 0})
null_mask = np.zeros((height, width), dtype=np.uint8)

track_ids_inframe = {}
custom_track_ids = {}
known_track_ids = []

# Function to calculate distance in pixels
def calculate_pixel_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# Function to calculate speed (assuming pixel distance and time interval)
def calculate_speed(pixel_distance, time_interval):
    return pixel_distance / time_interval  # Speed in pixels per second

# Function to process frames in batches
def process_frame_batch(frames):
    resized_frames = [cv2.resize(frame, (640 // 32 * 32, 480 // 32 * 32)) for frame in frames]
    frames_tensor = torch.from_numpy(np.stack(resized_frames)).permute(0, 3, 1, 2).float().to(device) / 255.0

    with torch.no_grad():
        batch_results = model(frames_tensor, device=device)

    return batch_results, resized_frames

import uuid

# Function to generate a custom track ID based on YOLO class, confidence, and a unique UUID
def generate_custom_track_id(label, confidence):
    return f"{label}_{confidence:.2f}_{uuid.uuid4()}"

# Function to track objects and draw segmentation polygons
def track_objects(frames, batch_results, frame_time):
    global camera_ip, previous_positions, fps, camera_id, track_ids_inframe, custom_track_ids, known_track_ids

    tracked_frames = []
    current_track_ids = []  # To keep track of the tracks currently in the frame

    for frame, result in zip(frames, batch_results):
        detections = []
        img_bin = []
        labels = []
        confs = []

        if result.masks:
            for mask, box in zip(result.masks.xy, result.boxes):
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                score = box.conf[0].item()
                label = model.names[int(box.cls[0])]
                detections.append([x1, y1, x2, y2, score])
                labels.append(label)
                confs.append(score)
                cv2.polylines(frame, [np.array(mask, dtype=np.int32)], isClosed=True, color=(0, 255, 0), thickness=1)
                cv2.putText(frame, f"{label} ({score:.2f})", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1)

        tracks = tracker.update(np.array(detections))

        for i, track in enumerate(tracks):
            track_id = int(track[4])
            x1, y1, x2, y2 = map(int, track[:4])

            # If the object is new, generate a custom track ID and store initial data
            if track_id not in custom_track_ids:
                custom_id = generate_custom_track_id(labels[i], confs[i])
                custom_track_ids[track_id] = {
                    "custom_track_id": custom_id,
                    "camera_id": camera_id,
                    "camera_ip": camera_ip,
                    "first_appearance": frame_time,  # Store first appearance time
                    "last_appearance": frame_time,   # Initialize last appearance time
                    "dbbox": [[x1, y1, x2, y2]],
                    "dlabel": [labels[i]],
                    "dconf": [confs[i]],
                }
            else:
                # Append the new frame data to the existing object in the dict
                custom_track_ids[track_id]["dbbox"].append([x1, y1, x2, y2])
                custom_track_ids[track_id]["dlabel"].append(labels[i])
                custom_track_ids[track_id]["dconf"].append(confs[i])
                custom_track_ids[track_id]["last_appearance"] = frame_time  # Update last appearance time

            # Add current track ID to the list of track IDs in the current frame
            current_track_ids.append(track_id)

            # Display the custom track ID on the frame
            cv2.putText(frame, f"ID: {custom_track_ids[track_id]['custom_track_id']}", (x1, y1 - 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 1)

            # Append the current frame to the tracked frames
            tracked_frames.append(frame)

    # Check for tracks that are no longer in the current frame (left the frame)
    tracks_left_frame = set(custom_track_ids.keys()) - set(current_track_ids)

    # Insert data into Redis for tracks that left the frame
    for track_id in tracks_left_frame:
        track_data = custom_track_ids[track_id]
        r.set(track_data['custom_track_id'], str(track_data))  # Insert into Redis as a string or JSON

        # Remove the track ID from the custom_track_ids since it left the frame
        del custom_track_ids[track_id]

    return tracked_frames, list(custom_track_ids.keys())


# Function to stream video and process frames in batches
def stream_process(camera_id, camera_ip, video_path, batch_size=8):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter("/home/annone/ai/data/output.mp4", fourcc, fps, (640,480))
    if not cap.isOpened():
        print(f"Error opening video file: {video_path}")
        return

    frames = []
    t1 = time.time()
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Record the current time in seconds for tracking purposes
        frame_time = time.time()

        frames.append(frame)

        if len(frames) >= batch_size:
            batch_results, resized_frames = process_frame_batch(frames)

            tracked_frames, track_id_list = track_objects(resized_frames, batch_results, frame_time)
            for tracked_frame in tracked_frames:
                cv2.imshow("Tracked Frame", tracked_frame)
                out.write(tracked_frame)
            frames = []
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    t2 = time.time()
    print(t2-t1)
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(custom_track_ids)

# Example usage
video_path = '/home/annone/ai/data/T-pole wrong way.mp4'
cam_ip = '127.0.0.1'
cam_id = "1"
stream_process(cam_id, cam_ip, video_path, batch_size=2)


cuda as Computation Device initiated

0: 480x640 4 autos, 3 cars, 1 man, 4 motorbike-riders, 1 pickup, 4 tractors, 31.0ms
1: 480x640 4 autos, 3 cars, 1 hatchback, 1 man, 3 motorbike-riders, 1 tractor, 31.0ms
Speed: 0.0ms preprocess, 31.0ms inference, 45.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 autos, 3 cars, 1 hatchback, 1 man, 3 motorbike-riders, 1 tractor, 5.1ms
1: 480x640 4 autos, 1 bike-rider, 3 cars, 1 man, 2 motorbike-riders, 1 tractor, 5.1ms
Speed: 0.0ms preprocess, 5.1ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 autos, 1 bike-rider, 3 cars, 1 man, 2 motorbike-riders, 1 tractor, 5.1ms
1: 480x640 4 autos, 2 cars, 1 man, 2 motorbike-riders, 1 pickup, 1 tractor, 5.1ms
Speed: 0.0ms preprocess, 5.1ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 autos, 2 cars, 1 man, 2 motorbike-riders, 1 pickup, 1 tractor, 5.1ms
1: 480x640 5 autos, 2 cars, 1 man, 4 motorbike-riders, 1 pickup, 5.1ms
Speed: 

In [ ]:
from collections import Counter, defaultdict

# 1. Mode-Based Approach: Top 3 most frequent labels
def top_3_label_mode(dlabel):
    label_counts = Counter(dlabel)
    top_3_labels = label_counts.most_common(3)
    return top_3_labels  # Returns label with its count

# 2. Weighted Confidence Sum Approach: Top 3 labels by cumulative confidence
def top_3_label_weighted_conf(dlabel, dconf):
    weighted_conf_dict = defaultdict(float)
    
    # Sum the confidences for each label
    for label, conf in zip(dlabel, dconf):
        weighted_conf_dict[label] += conf
    
    # Get top 3 labels with highest weighted confidence
    top_3_weighted = sorted(weighted_conf_dict.items(), key=lambda x: x[1], reverse=True)[:3]
    
    return top_3_weighted  # Returns label with its weighted confidence

# 3. Highest Confidence Approach: Top 3 labels with the highest individual confidence scores
def top_3_label_highest_conf(dlabel, dconf):
    # Combine labels and confidences into pairs and sort by confidence
    label_conf_pairs = sorted(zip(dlabel, dconf), key=lambda x: x[1], reverse=True)
    
    # Get top 3 based on confidence
    top_3_highest_conf = label_conf_pairs[:3]
    
    return top_3_highest_conf  # Returns label with its confidence

# Example usage
dlabel = ['tractor', 'hatchback', 'car', 'car', 'car', 'car', 'car', 'pickup', 'pickup', 'auto', 'auto', 'car', 
          'car', 'auto', 'auto', 'pickup', 'pickup', 'pickup', 'tractor', 'auto', 'auto', 'tractor', 'auto', 
          'tractor', 'car', 'car', 'tractor', 'tractor', 'tractor', 'pickup', 'motorbike-rider', 'pickup', 'car', 
          'tractor', 'car', 'car', 'auto']
dconf = [0.275, 0.274, 0.271, 0.333, 0.334, 0.443, 0.444, 0.392, 0.381, 0.429, 0.342, 0.356, 
         0.349, 0.500, 0.498, 0.387, 0.372, 0.463, 0.341, 0.618, 0.274, 0.350, 0.341, 0.331, 
         0.300, 0.279, 0.303, 0.391, 0.375, 0.465, 0.360, 0.466, 0.460, 0.434, 0.425, 0.482, 0.408]

# Get top 3 selections using different methods
top_3_mode = top_3_label_mode(dlabel)
top_3_weighted = top_3_label_weighted_conf(dlabel, dconf)
top_3_high_conf = top_3_label_highest_conf(dlabel, dconf)

# Print results
print("Top 3 labels (Mode-based):", top_3_mode)
print("Top 3 labels (Weighted confidence):", top_3_weighted)
print("Top 3 labels (Highest individual confidence):", top_3_high_conf)
